# DeepLeGATo++ Configuration - Colab L4
Configuration optimized for Google Colab L4 GPU (24GB VRAM, Ada Lovelace architecture)

**L4 Key specs:**
- 24GB GDDR6 VRAM
- FP16/BF16 support with good throughput
- ~120 TFLOPS FP16 Tensor Core
- More VRAM than T4 (16GB), less than A100 (40GB)

In [ ]:
import yaml
import os

In [ ]:
config = {
    "model": {
        "name": "DeepLeGATo++",
        "backbone": {
            "type": "swinv2_base_window8_256",  # Can use base model with 24GB
            "pretrained": True,
            "embed_dim": 128,
            "depths": [2, 2, 18, 2],
            "num_heads": [4, 8, 16, 32],
            "window_size": 7,
            "gradient_checkpointing": True,  # Saves ~30% VRAM, needed for base model on 24GB
            "drop_path_rate": 0.2
        },
        "npe": {
            "num_flow_layers": 16,
            "hidden_dim": 512,
            "num_transforms": 8,
            "num_blocks": 4,
            "dropout": 0.1
        },
        "num_params": 7  # mag, Re, n, q, PA, x, y
    },
    "training": {
        "batch_size": 32,  # 24GB allows 32 with grad checkpointing; effective 64 with accum
        "accumulation_steps": 2,  # Effective batch size: 64
        "max_epochs": 100,
        "optimizer": {
            "type": "AdamW",
            "learning_rate": 1.5e-4,  # Between T4 (1e-4) and A100 (2e-4), tuned for effective bs 64
            "weight_decay": 0.05,
            "betas": [0.9, 0.999]
        },
        "scheduler": {
            "type": "CosineAnnealingWarmRestarts",
            "T_0": 10,
            "T_mult": 2,
            "eta_min": 1.0e-6
        },
        "precision": "16-mixed",  # FP16 mixed precision, well-supported on L4
        "checkpoint_every_n_epochs": 1,
        "save_to_drive": True,
        "resume_from_latest": True,
        "early_stopping_patience": 15
    },
    "data": {
        "image_size": 256,  # 24GB can handle 256 with grad checkpointing
        "num_workers": 2,  # Colab has limited CPU cores
        "pin_memory": True,
        "prefetch_factor": 3,
        "priors": {
            "magnitude": [15.0, 28.0],
            "effective_radius": [0.1, 10.0],  # arcsec
            "sersic_index": [0.3, 8.0],
            "axis_ratio": [0.1, 1.0],
            "position_angle": [0.0, 180.0],  # degrees
            "center_offset": [-5.0, 5.0]  # pixels
        },
        "noise": {
            "sky_background": 0.01,
            "read_noise": 5.0,
            "gain": 2.5
        },
        "drive_base": "/content/drive/MyDrive/DeepLeGATo++",
        "num_train_samples": 200000,
        "num_val_samples": 20000,
        "generate_on_the_fly": True
    },
    "logging": {
        "use_wandb": True,
        "project_name": "DeepLeGATo++",
        "log_every_n_steps": 25,
        "log_images_every_n_epochs": 5
    },
    "validation": {
        "val_check_interval": 1.0,  # Every epoch
        "num_posterior_samples": 200  # For uncertainty estimation
    }
}

# Set derived paths
config["data"]["train_data_path"] = config["data"]["drive_base"] + "/data/train"
config["data"]["val_data_path"] = config["data"]["drive_base"] + "/data/val"
config["data"]["checkpoint_path"] = config["data"]["drive_base"] + "/checkpoints"
config["logging"]["log_dir"] = config["data"]["drive_base"] + "/logs"

print("L4 Configuration loaded successfully!")
print(f"  Backbone: {config['model']['backbone']['type']}")
print(f"  Image size: {config['data']['image_size']}")
print(f"  Batch size: {config['training']['batch_size']} x {config['training']['accumulation_steps']} accum = {config['training']['batch_size'] * config['training']['accumulation_steps']} effective")
print(f"  Precision: {config['training']['precision']}")
print(f"  Grad checkpointing: {config['model']['backbone']['gradient_checkpointing']}")
config

In [ ]:
# Save as YAML if needed
# with open('colab_l4.yaml', 'w') as f:
#     yaml.dump(config, f, default_flow_style=False)